In [4]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from PIL import Image
from sklearn.model_selection import train_test_split

def load_images_from_folder(folder):
    images = []
    labels = []
    for label, subfolder in enumerate(sorted(os.listdir(folder))):
        subfolder_path = os.path.join(folder, subfolder)
        for filename in os.listdir(subfolder_path):
            if filename.endswith(".BMP"):
                img_path = os.path.join(subfolder_path, filename)
                img = Image.open(img_path).convert('L')
                img = img.resize((16, 16))
                img_array = np.array(img) / 255.0
                images.append(img_array.flatten())
                labels.append(label)
    return np.array(images), np.array(labels)

# Load dataset
base_path = '/kaggle/input/socofing/socofing'  # Adjust this path
X, y = load_images_from_folder(base_path)
y = to_categorical(y, num_classes=20)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

2024-07-12 06:17:19.049706: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-12 06:17:19.049811: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-12 06:17:19.205605: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [5]:
print(X_train.shape) 
print(X_test.shape)  
print(y_train.shape) 
print(y_test.shape) 

(4800, 256)
(1200, 256)
(4800, 20)
(1200, 20)


In [6]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten

# Define the model
model = Sequential([
    Dense(128, activation='relu', input_shape=(256,)),  # 32*32 = 1024
    Dense(64, activation='relu'),
    Dense(20, activation='softmax')  # 20 classes
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test)) 
model.save('fingerprint_model.h5')

/opt/conda/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
 93/150 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.1216 - loss: 2.8526

I0000 00:00:1720765092.754671     121 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
W0000 00:00:1720765092.773869     121 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


124/150 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.1289 - loss: 2.8295

W0000 00:00:1720765093.533212     121 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


150/150 ━━━━━━━━━━━━━━━━━━━━ 4s 8ms/step - accuracy: 0.1357 - loss: 2.8121 - val_accuracy: 0.2042 - val_loss: 2.6012
Epoch 2/10
150/150 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.2160 - loss: 2.5195 - val_accuracy: 0.2583 - val_loss: 2.4086
Epoch 3/10
150/150 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.2660 - loss: 2.3276 - val_accuracy: 0.2667 - val_loss: 2.3715
Epoch 4/10
150/150 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.2840 - loss: 2.2644 - val_accuracy: 0.2833 - val_loss: 2.2782
Epoch 5/10
150/150 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.3043 - loss: 2.2004 - val_accuracy: 0.3017 - val_loss: 2.2168
Epoch 6/10
150/150 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.3086 - loss: 2.1755 - val_accuracy: 0.3017 - val_loss: 2.2024
Epoch 7/10
150/150 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.3294 - loss: 2.1509 - val_accuracy: 0.2900 - val_loss: 2.2021
Epoch 8/10
150/150 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.3196 - loss: 2.1457 - val_accuracy: 0.3158 - val_

In [7]:
!pip install tenseal

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


In [8]:
import tenseal as ts

# Create a TenSEAL context for the CKKS scheme
# Increase poly_modulus_degree to a higher value
context = ts.context(ts.SCHEME_TYPE.CKKS, poly_modulus_degree=16384, coeff_mod_bit_sizes=[60, 40, 40, 60])
context.global_scale = 2**40
context.generate_galois_keys()

def encrypt_weights(weights, context):
    encrypted_weights = [ts.ckks_vector(context, w.flatten().tolist()) for w in weights]
    return encrypted_weights

# Get model weights and biases
weights = model.get_weights()

# Encrypt weights
encrypted_weights = encrypt_weights(weights, context)

# Serialize encrypted weights
serialized_weights = [ew.serialize() for ew in encrypted_weights]
with open('encrypted_weights.bin', 'wb') as f:
    for sw in serialized_weights:
        f.write(sw)

The following operations are disabled in this setup: matmul, matmul_plain, enc_matmul_plain, conv2d_im2col.
If you need to use those operations, try increasing the poly_modulus parameter, to fit your input.


In [7]:
import tenseal as ts
import numpy as np

# Re-create or load the same TenSEAL context used for encryption
context = ts.context(ts.SCHEME_TYPE.CKKS, poly_modulus_degree=8192, coeff_mod_bit_sizes=[60, 40, 40, 60])
context.global_scale = 2**40
context.generate_galois_keys()

# Function to deserialize and decrypt weights
def decrypt_weights(serialized_weights, context):
    decrypted_weights = []
    for sw in serialized_weights:
        encrypted_vector = ts.ckks_vector_from(context, sw)
        decrypted_data = encrypted_vector.decrypt()
        decrypted_weights.append(np.array(decrypted_data))
    return decrypted_weights

# Load and deserialize encrypted weights
with open('encrypted_weights.bin', 'rb') as f:
    serialized_weights = []
    while True:
        # Adjust the chunk size to match the size of the serialized weights
        chunk = f.read(1024*1024)  # Read chunks of 1 MB; adjust size based on your serialization
        if not chunk:
            break
        serialized_weights.append(chunk)

# Decrypt weights
weights = decrypt_weights(serialized_weights, context)

# Since weights were flattened and serialized, they need to be reshaped to their original dimensions
original_shapes = [(32 * 32, 128), (128,), (128, len(class_names)), (len(class_names),)]
reshaped_weights = [w.reshape(shape) for w, shape in zip(weights, original_shapes)] 

ValueError: failed to parse CKKS stream

In [9]:
import os
import numpy as np
import tenseal as ts
from PIL import Image

def create_ckks_context():
    # Create a TenSEAL context for CKKS homomorphic encryption
    context = ts.context(ts.SCHEME_TYPE.CKKS, poly_modulus_degree=8192, coeff_mod_bit_sizes=[60, 40, 40, 60])
    context.global_scale = 2**40
    context.generate_galois_keys()
    return context

def encrypt_image(image_array, context):
    # Encrypt an image array using CKKS
    encrypted_image = ts.ckks_vector(context, image_array)
    return encrypted_image

def load_and_encrypt_images(folder, context):
    encrypted_images = []
    labels = []
    for label, subfolder in enumerate(sorted(os.listdir(folder))):
        subfolder_path = os.path.join(folder, subfolder)
        for filename in os.listdir(subfolder_path):
            if filename.endswith(".BMP"):
                img_path = os.path.join(subfolder_path, filename)
                img = Image.open(img_path).convert('L')
                img = img.resize((16, 16))  # Resize images
                img_array = np.array(img) / 255.0  # Normalize images
                encrypted_img = encrypt_image(img_array.flatten(), context)
                encrypted_images.append(encrypted_img.serialize())  # Serialize encrypted data
                labels.append(label)
    return encrypted_images, np.array(labels)

# Create the CKKS encryption context
context = create_ckks_context()

# Load and encrypt dataset
base_path = '/kaggle/input/socofing/socofing'  # Adjust this path to your dataset
encrypted_images, y = load_and_encrypt_images(base_path, context)

# Example of how to save encrypted images
import pickle
with open('encrypted_images.pkl', 'wb') as f:
    pickle.dump(encrypted_images, f)  # Save the serialized encrypted images